<a href="https://colab.research.google.com/github/albert-yue/gcn-explanability/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My\ Drive/gcn_explainability

/content/gdrive/My Drive/gcn_explainability


In [0]:
! git commit -m "Update"

[master 6d4a833] Update
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite gcn.ipynb (97%)


In [0]:
! git pull

Auto-merging gcn.ipynb
CONFLICT (content): Merge conflict in gcn.ipynb
Automatic merge failed; fix conflicts and then commit the result.


In [0]:
%ls

data/  gcn.ipynb  logs.pt  notebooks/  README.md  src/


In [0]:
from src.models.gcn import GCN
import torch
import numpy as np
import scipy.sparse as sparse
import scipy
from src.data import Corpus, get_data, get_labels, get_vocabulary
from src.utils import load_sparse_tensor
import torch.nn as nn
from torch.autograd import Variable
from src.preprocessing import build_adj_matrix, normalize_adj

logs = (200, 0.5, 41664, 23, 'logs.pt')
ng20 = (200, 0.5, 80035, 20, 'gcn_20ng_full.pt', 61189, 18846, 11314,7532)
hidden_size,dropout,num_vertices,num_labels,path, num_words, num_docs, training_docs, test_docs = ng20


vocabulary = get_vocabulary('data/20ng-vocabulary.txt')
labels = get_labels('data/20ng-labels.txt')
test_corpus = get_data('data/test-20news-clean.txt', labels)
test_adj_matrix = load_sparse_tensor('data/20ng_full_adj_matrix_norm.pt')
train_corpus = get_data('data/train-20news-clean.txt', labels)

In [0]:
def matrices(path, adj, index, label):
  model = GCN(num_vertices, hidden_size, num_labels, dropout=dropout)
  model.load_state_dict(torch.load(path))
  model.eval()
  loss_fn = nn.CrossEntropyLoss()
  model.zero_grad()
  output = model(adj)
  loss = loss_fn(output[-index:-index+1,:], label)
  loss.retain_grad()
  loss.backward()
  weight_matrices = []
  gradients = []
  for n, p in model.named_parameters():
    print(n)
    weight_matrices.append(p.detach().numpy())
    gradients.append(p.grad.numpy())
    print(p.grad.shape)
  return weight_matrices, gradients, output

In [0]:
def grad_cam_avg(adj, weight_matrices, gradients):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  grad_cams = []
  for l in range(2):
    grad_cam = []
    grad_cams.append(grad_cam)
    alphas = np.mean(gradients[l], axis = 0)
    for n in range(num_vertices):
      grad_cam.append(np.maximum(0, np.dot(alphas, F[l][n, :])))

  grad_cam_avgs = np.mean(grad_cams, axis=0)
  return grad_cams, grad_cam_avgs

In [0]:
def convert_to_words(top_words, vocabulary, labels, train_corpus):
  words = []
  for word_idx in top_words:
      if word_idx < num_words:
        words.append(vocabulary[word_idx])
      elif word_idx < num_words+training_docs:
        words.append(labels[train_corpus[word_idx-num_words].label])
      else:
        words.append(labels[test_corpus[word_idx-num_words-training_docs].label])
  return words

In [8]:

#grad_cam for specific docs
indices = [2, 7468]
gradient_test = []
for i in indices:
  single_corpus = Corpus([test_corpus.data[-i]])
  adj = build_adj_matrix(single_corpus, vocabulary, num_docs, training_docs+i)
  adj = normalize_adj(adj)
  weight_matrices, gradients, output = matrices(path, adj, i, single_corpus.labels())
  gradient_test.append(gradients)
  grad_cams, grad_cam_avgs = grad_cam_avg(adj, weight_matrices, gradients)
  print(test_corpus.data[-i].text)
  print(labels[test_corpus.data[-i].label])
  print(labels[np.argmax(output[-i,:].detach().numpy())])
  for name, grad_cam_vals in [('layer1', grad_cams[0]), ('layer2', grad_cams[1]), ('overall', grad_cam_avgs)]:
    print(name+":\n")
    top_words_idx = np.asarray(grad_cam_vals).argsort()[-10:]
    top_words = convert_to_words(reversed(top_words_idx), vocabulary, labels, train_corpus)
    for word in top_words:
      print(word)
  
  max_grads = np.asarray(np.linalg.norm(np.matmul(gradients[0], gradients[1]),axis=1)).argsort()[-10:]
  print(convert_to_words(reversed(max_grads), vocabulary, labels, train_corpus))



Building word frequencies per doc



Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities


...


Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities



layer1.weight
torch.Size([80035, 200])
layer2.weight
torch.Size([200, 20])
['eng', 'sun', 'com', '<unk>', 'subject', 'serbs', 'get', 'ammunition', 'organization', 'sun', 'microsystems', 'inc', 'lines', '<unk>', 'distribution', 'world', 'reply', 'eng', 'sun', 'com', 'nntp', 'posting', 'host', 'trying', 'follow', 'current', 'conflict', 'former', 'yugoslavia', 'one', 'thing', 'cannot', 'figure', 'serbs', 'croats', 'get', 'weapons', 'etc', 'run', 'ram']
talk.politics.mideast
talk.politics.guns
layer1:

talk.religion.misc
sci.electronics
rec.autos
rec.autos
comp.graphics
talk.politics.guns
comp.sys.mac.hardware
talk.politics.guns
talk.politics.guns
sci.space
layer2:

rec.sport.baseball
soc.religion.christian
supercharger
sci.space
jth
linguists
imgarc
lillehammer
talk.politics.misc
ffbf
overall:

rec.sport.baseball
soc.religion.christian
talk.politics.misc
supercharger
fainted
linguists
sci.space
imgarc
lillehammer
endometriosis
['talk.politics.mideast', 'talk.religion.misc', 'toxoids', 'm

In [0]:


total = 0
max_num = 10
for i in range(1, 7500):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

total = 0
for i in range(1, 7500):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    total += 1
print(total/7500)

total = 0
for i in range(7500, 1, -1):
  if test_corpus.data[-i].label != np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

2
3
4
7
8
9
10
11
12
13
0.7456
7500
7491
7490
7488
7486
7484
7478
7474
7468
7462


In [0]:
def p_carrot_val(carrot, p, weight_matrix):
  print(carrot.shape)
  p_carrot = np.matmul(np.maximum(0, weight_matrix), carrot.T)
  p_carrot = p_carrot / np.sum(p_carrot, axis= 0)
  p_carrot = np.matmul(p_carrot, p)
  return p_carrot

def p_F_val(v, F, p_carrot):
  print(F.shape)
  p_F = v.dot(F)
  p_F = p_F / np.sum(p_F, axis= 0)
  p_F = np.matmul(p_F, p_carrot.T)
  return p_F

def EB(adj, weight_matrices, gradients, output):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  F_carrot = [V.dot(F_1), V.dot(F_2)]
  p_F3 = output.detach().numpy()
  p_carrot_F2 = p_carrot_val(F_carrot[1], p_F3, weight_matrices[1])
  p_F2 = p_F_val(V, F[1], p_carrot_F2)
  print(F_carrot[0].shape)
  print(weight_matrices[0].shape)
  return p_F2
  p_carrot_F1 = p_carrot_val(F_carrot[0], p_F2, weight_matrices[0])
  p_F1 = p_F_val(V, F[0], p_carrot_F1)
  p_F = [p_F1, p_F2, p_F3]
  p_carrot_F = [p_carrot_F1, p_carrot_F2]
  eb_vals = np.mean(p_F1, axis=1)
  return eb_vals


indices = [14]
gradient_test = []
for i in indices:
  single_corpus = Corpus([test_corpus.data[i]])
  adj = build_adj_matrix(single_corpus, vocabulary, num_docs, 1)
  adj = normalize_adj(adj)
  weight_matrices, gradients, output = matrices(path, adj)
  eb_vals = EB(adj, weight_matrices, gradients, output)
  print(eb_vals.shape)


Building word frequencies per doc



Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities



layer1.weight
torch.Size([80035, 200])
layer2.weight
torch.Size([200, 20])
(80035, 20)
(80035, 20)
(80035, 200)
(80035, 200)
(80035, 200)
